In [1]:
# Load documents

from langchain.document_loaders import WebBaseLoader

# loader = WebBaseLoader("https://anelmusic13.medium.com/how-to-score-top-3-in-kaggles-titanic-machine-learning-from-disaster-competition-13d056e262b1")

In [1]:
from langchain.document_loaders import PyPDFLoader

In [ ]:

# loader = PyPDFLoader("/Users/abhinay/Downloads/Real-Time_Detection_of_DNS_Exfiltration_and_Tunneling_from_Enterprise_Networks-1.pdf")
# pages = loader.load_and_split()

In [2]:
loader = PyPDFLoader("/Users/abhinay/Downloads/Real-Time_Detection_of_DNS_Exfiltration_and_Tunneling_from_Enterprise_Networks-1.pdf", extract_images=True)
pages = loader.load()

In [3]:
# Split documents

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
splits = text_splitter.split_documents(pages)

In [4]:
key = "come-on"

In [5]:
# Embed and store splits
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

embeddings  = OpenAIEmbeddings(model="text-embedding-ada-002", openai_api_key=key)
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)
retriever = vectorstore.as_retriever()

In [6]:
from langchain.prompts.prompt import PromptTemplate

# Define a valid namespace with a prompt template
prompt_namespace = {
    'id': ['langchain', 'prompts', 'prompt', 'PromptTemplate'],
    'lc': 1,
    'type': 'constructor',
    'kwargs': {
        'template': "You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:",
        'input_variables': ['question', 'context'],
        'template_format': 'f-string'
    }
}

# Create an instance of the prompt template
prompt_template = PromptTemplate(**prompt_namespace['kwargs'])


In [7]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, openai_api_key=key)

In [8]:
rag_prompt = PromptTemplate(**prompt_namespace['kwargs'])

In [9]:
from langchain.schema import runnable
# langchain.schema.runnable import RunnablePassthrough

rag_chain = {"context": retriever, "question": runnable.RunnablePassthrough()} | rag_prompt | llm

In [10]:
rag_chain.invoke("What were the final parameters used in the isolationForest model")

AIMessage(content='The final parameters used in the isolationForest model were: number of trees = 2, height limit of trees = 18, and contamination rate = 2%.')

In [11]:
rag_chain.invoke("Describe what was there in the table 3")

AIMessage(content='Table 3 shows the rate of detection for the benign and "Others" instances in the two networks. It indicates that 98% of benign instances are correctly detected as normal during both cross-validation (Days 1-4) and testing (Days 5-7) phases. However, the specific details of the table are not provided in the given context.')

In [12]:
rag_chain.invoke("summarize the paper in paragraph")

AIMessage(content='The paper presents a real-time mechanism for detecting exfiltration and tunneling of data over DNS at the enterprise edge. It develops and trains a machine learning algorithm to detect anomalies in DNS queries using a benign dataset of top-ranked primary domains from two enterprise networks. The authors focus on attributes of fully qualified domain names to detect DNS tunneling.')

In [15]:
rag_chain.invoke("describe figure 2 along with statistics")

AIMessage(content='Figure 2 shows the value of attributes for detected instances (on the left) versus undetected instances (on the right) using the machine for the Research Institute. The undetected instances were shorter in total length and average label length compared to the detected instances. However, there is a fair overlap of value range between detected and undetected instances.')

In [17]:
rag_chain.invoke("explain about the average time complexity of their scheme based on the table 5 in simple terms")

AIMessage(content='The average time complexity of their scheme, as shown in Table 5, is 54µsec for extracting attributes, 746µsec for detecting anomalies, and a total time of 800µsec per query name. This means that their scheme can process about 1250 DNS queries per second.')

In [21]:
rag_chain.invoke("what were the attributes derived from the DNS and what were the attributes from FQFN, based on the output of this query form a list of features")

AIMessage(content='The attributes derived from the DNS include characters count, entropy of string, and length of discrete labels in the query name. The attributes from FQDN (Fully Qualified Domain Name) include the total count of characters in FQDN, which is important in determining if the query name carries embedded information for an outside host. The attributes are derived from individual DNS query packets and are independent of time-series characteristics of queried domains or hosts.')

In [22]:
rag_chain.invoke("can you describe steps to implement the model")

AIMessage(content='The steps to implement the model include: (1) cross-validating and testing the accuracy of the trained model for benign instances, (2) testing the detection rate for malicious DNS queries generated using the tool, and (3) quantifying the performance in real-time on live 10 Gbps traffic streams from the two organizations.')